In [ ]:
from array import *
import os
import pandas as pd
import requests
import sqlalchemy as db

%load_ext sql
%load_ext dotenv

%sql postgresql://$POSTGRES_USER:$POSTGRES_PWD@db/registry

conn_str = 'postgresql://%s:%s@db/registry' % (os.getenv("POSTGRES_USER"), os.getenv("POSTGRES_PWD"))
engine = db.create_engine(conn_str)
connection = engine.connect()

In [ ]:
THANOS_TOKEN = os.environ['THANOS_TOKEN']
THANOS_URL = 'https://thanos-querier-openshift-monitoring.apps.silver.devops.gov.bc.ca/api/v1/query' #os.environ['THANOS_URL']

headers = { 'Content-type': 'application/json', 'Authorization': 'Bearer %s' % THANOS_TOKEN } 
querystring = 'query=sum({__name__=~"namespace_cpu:kube_pod_container_resource_requests:sum|namespace:container_cpu_usage:sum|namespace_cpu:kube_pod_container_resource_limits:sum", namespace=~"%s.*"} offset 1w) BY (__name__)'
usage = {}

result = %sql SELECT project_set_id FROM project WHERE archived = false AND cluster = 'SILVER';

for n in list(map(lambda r: r[0], result)):
  p = querystring % n

  response = requests.get(THANOS_URL, headers = headers, params = p)
  if response.status_code != 200:
    quit()

  result_as_json = pd.read_json(response.text)
  
  try:
    # print(response.text)
    usage[n] = {}
    for r in result_as_json.data['result']:
      usage[n][r['metric']['__name__']] = round(float(r['value'][1]), 3)
    if len(usage[n].keys()) == 0:
      del usage[n]
    
  except Exception as e:
    print(e)


The keys to the dictionary will be:
- namespace:container_cpu_usage:sum
- namespace_cpu:kube_pod_container_resource_requests:sum
- namespace_cpu:kube_pod_container_resource_limits:sum

In [ ]:
conn_str = 'postgresql://app_api_704fkrus:bacon@db/registry'
engine = db.create_engine(conn_str)
connection = engine.connect()

meta_data = db.MetaData()
db.MetaData.reflect(meta_data, bind=engine)
resource_usage = meta_data.tables['resource_usage']

for key in usage:
  try:
    u = usage[key]['namespace:container_cpu_usage:sum']
    r = usage[key]['namespace_cpu:kube_pod_container_resource_requests:sum']
    l = usage[key]['namespace_cpu:kube_pod_container_resource_limits:sum']
    
    # print (key, u, r, l, round(u / r * 100, 2))

    stmt = db.insert(resource_usage).values(
        project_set_id=key,
        container_cpu_usage_sum=u,
        container_cpu_requests_sum=r,
        container_cpu_limit_sum=l)
    
    # print(stmt)
    connection.execute(stmt)
  except Exception as e:
      print(e)

connection.commit()